In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
device = 'cuda'

## Exploring the data

In [3]:
import pandas as pd

In [4]:
raw_train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
raw_test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [5]:
len(raw_train_df)

7613

In [6]:
raw_train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
raw_test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
duplicates_df = raw_train_df[raw_train_df.text.duplicated(keep=False)].sort_values('text')
duplicates_df.head()

,id,keyword,location,text,target
4290,6094,hellfire,"Jubail IC, Saudi Arabia.",#Allah describes piling up #wealth thinking it...,0
4299,6105,hellfire,?????? ??? ?????? ????????,#Allah describes piling up #wealth thinking it...,0
4312,6123,hellfire,?????? ???? ??????,#Allah describes piling up #wealth thinking it...,1
6363,9095,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
6373,9107,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1


## Clean the data

As we've seen, there are dosens of duplicated tweets. Moreover, their labes are sometimes contradictory.

In [9]:
duplicates_df[:20]

,id,keyword,location,text,target
4290,6094,hellfire,"Jubail IC, Saudi Arabia.",#Allah describes piling up #wealth thinking it...,0
4299,6105,hellfire,?????? ??? ?????? ????????,#Allah describes piling up #wealth thinking it...,0
4312,6123,hellfire,?????? ???? ??????,#Allah describes piling up #wealth thinking it...,1
6363,9095,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
6373,9107,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
6377,9113,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
6378,9114,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
6392,9135,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
6366,9098,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
2828,4064,displaced,NaN,#KCA #VoteJKT48ID 12News: UPDATE: A family of ...,1


In [10]:
duplicated_tweets = list(duplicates_df.text)

In [11]:
len(duplicated_tweets)

179

Now search for contradictions

In [12]:
contradictory_tweets = set()
for tweet in duplicated_tweets:
    if len(set(duplicates_df[duplicates_df['text'] == tweet].target)) > 1:
        contradictory_tweets.add(tweet)

contradictory_tweets = list(contradictory_tweets)
print(len(contradictory_tweets), type(contradictory_tweets))

18 <class 'list'>


We're going to label those manually

In [13]:
for i, tweet in enumerate(contradictory_tweets):
    print(i, tweet, '\n')

0 In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism! 

1 #Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect 

2 #foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption 

3 .POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4 

4 Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam! 

5 Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife 

6 I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ?????? 

7 To fight bioterrorism sir. 

8 Caution: breathing may be hazardous to your health. 

9 Who is bringing the tornadoes and floods. Who is bringing the climate change. God is afte

In [14]:
no_duplicates_train_df = raw_train_df.drop_duplicates('text')
print(len(no_duplicates_train_df))

7503


In [15]:
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[0], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[1], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[2], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[3], 'target'] = 0 
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[4], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[5], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[6], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[7], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[8], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[9], 'target'] = 1
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[10], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[11], 'target'] = 1
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[12], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[13], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[14], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[15], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[16], 'target'] = 0
no_duplicates_train_df.loc[no_duplicates_train_df['text'] == contradictory_tweets[17], 'target'] = 1

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## Create the Datasets

In [16]:
import torch
from torch.utils.data import Dataset
from transformers import RobertaTokenizer,RobertaModel

from tqdm import tqdm
from copy import deepcopy
import random

In [17]:
model_checkpoint = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

In [18]:
class LabeledDisasterTweetsDataset(Dataset):
    def __init__(self, 
                 df, 
                 tokenizer, 
                 max_tweet_len=256, 
                 tweet_column='text',
                 location_column='location', 
                 keyword_column='keyword', 
                 target_column='target', 
                 device='cpu'):
        
        self._data = []
        
        for index, row in tqdm(df.iterrows(), total=len(df), ncols=70):
            tokenized = tokenizer(row[tweet_column], padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self._data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target_column]).to(device)))
        
        print('NOTE: you can safely delete the dataframe')
        
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, i):
        """return: ((input_ids, attention_mask), target)"""
        return self._data[i] 
    
    def train_valid_split(self, train_fraction=.9, shuffle=True):
        num_train_examples = int(len(self) * train_fraction)
        
        train_dataset = deepcopy(self)
        
        if shuffle:
            random.shuffle(train_dataset._data)
        
        valid_dataset = deepcopy(train_dataset)
        train_dataset._data = train_dataset._data[:num_train_examples]
        valid_dataset._data = valid_dataset._data[num_train_examples:]
        
        print('NOTE: you can safely delete parent dataset. The train|valid split is {}|{}.'.format(len(train_dataset), len(valid_dataset)))
        
        return train_dataset, valid_dataset
    
    
class UnlabeledDisasterTweetsDataset(Dataset):
    def __init__(self, 
                 df, 
                 tokenizer, 
                 max_tweet_len=256, 
                 tweet_column='text', 
                 id_column='id', 
                 device='cpu'):
        
        self._data = []
        
        for index, row in tqdm(df.iterrows(), total=len(df), ncols=70):
            tokenized = tokenizer(row[tweet_column], padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self._data.append((torch.tensor(row[id_column]).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))
        
        print('NOTE: you can safely delete the dataframe')
        
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, i):
        """return: (id, (input_ids, attention_mask))"""
        return self._data[i] 

In [19]:
dataset = LabeledDisasterTweetsDataset(no_duplicates_train_df, tokenizer, device=device)

100%|████████████████████████████| 7503/7503 [00:09<00:00, 778.57it/s]

NOTE: you can safely delete the dataframe


In [20]:
train_dataset, valid_dataset = dataset.train_valid_split()

NOTE: you can safely delete parent dataset. The train|valid split is 6752|751.


In [21]:
eval_dataset = UnlabeledDisasterTweetsDataset(raw_test_df, tokenizer, device='cuda')

100%|███████████████████████████| 3263/3263 [00:02<00:00, 1129.79it/s]

NOTE: you can safely delete the dataframe


In [22]:
print(train_dataset[0],'\n\n', train_dataset[-1],'\n\n', valid_dataset[0])

((tensor([    0,  1039, 24009,   139,  1990, 14601,   261,   787,   565,  1942,
        38469,    45,     7, 16195,  2990,    53,    25,    10, 33954, 35832,
          740, 13714,  8772,    38,    64,  4559,    42,    25,  1528,   131,
        40913, 28117,  5457,    73,  5214,  5100,    73,  5605,    13,   390,
            4,     2,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1

In [23]:
print(eval_dataset[0])

(tensor(0, device='cuda:0'), (tensor([   0, 6785, 1102,   10, 6587,  512, 2058,    2,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          

## Defining the model

In [24]:
class RobertaClassifier(torch.nn.Module):
    def __init__(self, n_classes=2, head_dropout=.3, head_activation=torch.nn.ReLU, model_checkpoint='roberta-base'):
        super().__init__()
        
        self.Roberta = RobertaModel.from_pretrained(model_checkpoint)
        
        self.Head = torch.nn.Sequential(
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(768, n_classes)
        )
        
    def forward(self, input_ids, attention_mask=None, *argv):
        res = self.Roberta(input_ids=input_ids, attention_mask=attention_mask)
        res = res['pooler_output']
        res = self.Head(res)
        return res
    
    def parameters_num(self):
        return sum(p.numel() for p in self.parameters())

In [25]:
model = RobertaClassifier()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
model.to(device)

RobertaClassifier(
  (Roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [27]:
model(torch.tensor([[1]], device=device))

tensor([[-0.1332,  0.0338]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [28]:
print('Our Roberta has {:,d} parameters...'.format(model.parameters_num()))

Our Roberta has 124,647,170 parameters...


## Fine-tuning

In [29]:
from torch.utils.data import DataLoader

In [30]:
batch_size = 8

In [31]:
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [33]:
def train(model, 
          train_dataloader, 
          valid_dataloader, 
          steps, 
          optimizer,
          blind_steps=None,
          loss_fn=torch.nn.BCELoss(),
          main_metric=('f1', f1_score), 
          additional_metrics=[],
          filepath='model_sd.pt',
          load_best=True):
    
    if blind_steps == None:
        blind_steps = len(train_dataloader) // 4
    
    def evaluate():  # the first score returned is the main
        model.eval()
        
        y_trues = []
        y_hats = []
        
        loss = 0
        k = 0
        
        with torch.no_grad():
            for batch in valid_dataloader:
                
                (ids, mask), y_true = batch
                hots = torch.nn.functional.one_hot(y_true, 2).to(device, torch.float)
                y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

                loss += float(loss_fn(y_hat, hots))
                k += 1
                
                for i in range(y_true.shape[0]):
                    y_trues.append(int(y_true[i]))
                    y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)
        
        scores = [(main_metric[0], main_metric[1](y_trues, y_hats))]
        
        for metric in additional_metrics:
            scores.append((metric[0], metric[1](y_trues, y_hats)))        
        
        model.train()
        return scores + [('valid_loss', loss/k)]
    
    
    def render_scores(scores, step, best=None):
        print('{:05d} steps'.format(step), end=' ')
        
        for score in scores:
            print("| {}: {:.3f}".format(*score), end=' ')
            
        if best != None:
            print('| best_score: {:.3f}'.format(best))
    
    train_iter = iter(train_dataloader)
    model.train()
    
    scores = evaluate()
    render_scores(scores, 0)
    best_score = .7 # threshold
    torch.save(model.state_dict(), filepath)
    
    epoch_loss = 0
    k = 0
    
    for step in tqdm(range(steps)):
        
        
        try:
            batch = next(train_iter)
        except:
            train_iter = iter(train_dataloader)
            batch = next(train_iter)

        (ids, mask), y_true = batch

        model.zero_grad()
        y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

        hots = torch.nn.functional.one_hot(y_true, 2).to(device, torch.float)
        loss = loss_fn(y_hat, hots)
        loss.backward()

        optimizer.step()

        epoch_loss += float(loss)
        k+=1

        if (step + 1) % blind_steps == 0:
            scores = evaluate() + [('train_loss', epoch_loss/k)]
            if scores[0][1] > best_score:
                best_score = scores[0][1]
                torch.save(model.state_dict(), filepath)
            render_scores(scores, step + 1, best=best_score)
            epoch_loss = 0
            k = 0
                
    if load_best:
        model.load_state_dict(torch.load(filepath))

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

In [35]:
train(model, 
      train_dataloader, 
      valid_dataloader, 
      11000, 
      optimizer, 
      blind_steps=250, 
      additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)])

00000 steps | f1: 0.342 | precision: 0.410 | recall: 0.293 | accuracy: 0.523 | valid_loss: 0.693 

  2%|▏         | 249/11000 [00:53<38:48,  4.62it/s]/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  2%|▏         | 250/11000 [01:00<6:05:47,  2.04s/it]

00250 steps | f1: 0.000 | precision: 0.000 | recall: 0.000 | accuracy: 0.578 | valid_loss: 0.673 | train_loss: 0.686 | best_score: 0.700


  5%|▍         | 500/11000 [02:01<6:51:32,  2.35s/it]

00500 steps | f1: 0.781 | precision: 0.786 | recall: 0.776 | accuracy: 0.816 | valid_loss: 0.457 | train_loss: 0.630 | best_score: 0.781


...

10500 steps | f1: 0.782 | precision: 0.767 | recall: 0.798 | accuracy: 0.812 | valid_loss: 0.540 | train_loss: 0.211 | best_score: 0.805


 98%|█████████▊| 10750/11000 [43:17<08:29,  2.04s/it]

10750 steps | f1: 0.779 | precision: 0.773 | recall: 0.785 | accuracy: 0.812 | valid_loss: 0.532 | train_loss: 0.221 | best_score: 0.805


100%|██████████| 11000/11000 [44:17<00:00,  4.14it/s]

11000 steps | f1: 0.777 | precision: 0.778 | recall: 0.776 | accuracy: 0.812 | valid_loss: 0.535 | train_loss: 0.208 | best_score: 0.805


In [36]:
train(model, 
      train_dataloader, 
      valid_dataloader, 
      5000, 
      optimizer, 
      blind_steps=250, 
      additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)])

00000 steps | f1: 0.805 | precision: 0.832 | recall: 0.779 | accuracy: 0.840 | valid_loss: 0.424 

  5%|▌         | 250/5000 [01:01<3:04:59,  2.34s/it]

00250 steps | f1: 0.798 | precision: 0.798 | recall: 0.798 | accuracy: 0.830 | valid_loss: 0.435 | train_loss: 0.283 | best_score: 0.798


 10%|█         | 500/5000 [02:01<2:32:49,  2.04s/it]

00500 steps | f1: 0.797 | precision: 0.802 | recall: 0.792 | accuracy: 0.830 | valid_loss: 0.439 | train_loss: 0.273 | best_score: 0.798


...

04500 steps | f1: 0.788 | precision: 0.803 | recall: 0.773 | accuracy: 0.824 | valid_loss: 0.518 | train_loss: 0.210 | best_score: 0.798


 95%|█████████▌| 4750/5000 [19:08<08:29,  2.04s/it]

04750 steps | f1: 0.787 | precision: 0.792 | recall: 0.782 | accuracy: 0.822 | valid_loss: 0.524 | train_loss: 0.215 | best_score: 0.798


100%|██████████| 5000/5000 [20:08<00:00,  4.14it/s]

05000 steps | f1: 0.786 | precision: 0.790 | recall: 0.782 | accuracy: 0.820 | valid_loss: 0.533 | train_loss: 0.219 | best_score: 0.798


## Evaluate

In [37]:
def evaluate(model, valid_dataloader, metrics=[('f1', f1_score),('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)]):
    model.eval()

    y_trues = []
    y_hats = []

    with torch.no_grad():
        for batch in valid_dataloader:

            (ids, mask), y_true = batch
            y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

            for i in range(y_true.shape[0]):
                y_trues.append(int(y_true[i]))
                y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

    scores = []

    for metric in metrics:
        scores.append((metric[0], metric[1](y_trues, y_hats)))        

    model.train()
    return scores

In [38]:
evaluate(model, valid_dataloader)

[('f1', 0.7981072555205049),
 ('precision', 0.7981072555205048),
 ('recall', 0.7981072555205048),
 ('accuracy', 0.829560585885486)]

In [39]:
def classify(tweet):
    ids = torch.tensor(tokenizer(tweet)['input_ids'])[None].to(device)
    return model.forward(input_ids=ids)

In [40]:
classify("Over 2 hundred families have become victims of my cookie-shop iniciative")

tensor([[ 1.1673, -1.1818]], device='cuda:0', grad_fn=<AddmmBackward0>)

## Make predictions

In [41]:

predictions_df = pd.DataFrame()
for i, (ids, mask) in tqdm(eval_dataset):
    pred = model(input_ids=ids[None], attention_mask=mask[None])[0]
    y_hat = 1 if pred[0] < pred[1] else 0
    r = [int(i), y_hat]
    predictions_df = pd.concat([predictions_df, pd.DataFrame(np.array(r)[None,:], columns=['id', 'target'])])


100%|██████████| 3263/3263 [00:43<00:00, 75.19it/s]


In [42]:
predictions_df.target = predictions_df.target.astype(int)
predictions_df.id = predictions_df.id.astype(int)
predictions_df.head(20)

,id,target
0,0,1
0,2,1
0,3,1
0,9,1
0,11,1
0,12,1
0,21,0
0,22,0
0,27,0
0,29,0


In [43]:
predictions_df.to_csv('submission.csv', index=False)
print('Done!')

Done!
